# An example of one time (one iteration) crossover and mutation on population used in Genetic Algorithm

In [28]:
import numpy as np

pop_size = 10
mutation_rate = 0.3
crossover_rate = 0.7
num_variables = 2
lower_bound = -5
upper_bound = +5

population = np.random.uniform(low=lower_bound, high=upper_bound, size=(pop_size, num_variables))

# fitness value for each individual (or chromosome OR a set of decision variables) must be computed by the objective funtion
# but here to just simpilify we assign a random number for each individual
fitness = np.random.uniform(low=0, high=10, size=(pop_size, 1))

print("Population:\n",population)
print("\nAssume these are the values computed by objective function for each individual:\n",fitness)


Population:
 [[ 2.29463618 -1.2461085 ]
 [-0.24739641  1.78155216]
 [ 1.99307439  4.40131682]
 [-0.79778746  2.82777041]
 [ 4.6496967   3.33868319]
 [-4.32155109 -0.55518814]
 [ 1.58053494  3.83618539]
 [ 1.98283975  3.10711972]
 [-3.64825719  1.27095742]
 [ 1.86410832 -4.32261067]]

Assume these are the values computed by objective function for each individual:
 [[7.91149628]
 [0.00823317]
 [7.66235049]
 [4.75095812]
 [1.21649746]
 [3.87425949]
 [5.29286533]
 [3.1937971 ]
 [5.15631021]
 [2.29502434]]


We want to minimize the objective function. So the more fitness values is, the less possibility it has to be the solution. That's why I do inverse of fitness. Deviding by `+1e-6` is to make sure we won't have division by zero.

In [29]:
fitness_probs = 1 / (fitness + 1e-15)  # Adding a small constant to avoid division by zero

# Normalize the probabilities
fitness_probs /= fitness_probs.sum()

print("The probility of individuals:\n",fitness_probs)

The probility of individuals:
 [[0.00101832]
 [0.97841489]
 [0.00105143]
 [0.00169575]
 [0.00662264]
 [0.00207948]
 [0.00152213]
 [0.00252253]
 [0.00156244]
 [0.00351039]]


I use `np.random.choice` function to pick the stronger individuals (the ones with higher probability are more likely to survive, althogh, there is a slight chance that individuals with less probability are going to survive too).

In [30]:
# Select parents to crossover based on their probabilities
selected_indices = np.random.choice(pop_size, size=pop_size, p=fitness_probs[:, 0])

# Create the new population with selected parents
new_population = population[selected_indices, :]

print("The index of individuals (chromosomes) that survived:\n",selected_indices)

The index of individuals (chromosomes) that survived:
 [1 1 1 0 1 1 1 1 1 1]


**IMPORTANT:** Every time you run this part a new set of indexes will be picked. But the ones you see more frequently are those which have higher probability in `fitness_probs`. Then I created `new_population` based on `selected_indices` holding the individuals with higher probability to be the best solution. And these survived individuals are going to make children and mutate to make the next generation!

In [31]:
print("New population:\n",new_population)

New population:
 [[-0.24739641  1.78155216]
 [-0.24739641  1.78155216]
 [-0.24739641  1.78155216]
 [ 2.29463618 -1.2461085 ]
 [-0.24739641  1.78155216]
 [-0.24739641  1.78155216]
 [-0.24739641  1.78155216]
 [-0.24739641  1.78155216]
 [-0.24739641  1.78155216]
 [-0.24739641  1.78155216]]


Here, every two parents in `new_population` will make two new kids! carrying the same gens that their parents had. Mind you that there is a `crossover_rate` percent chance that two parents would be able to mate. Others will remain untouched. They stay single or you can say they don't want to be in a serious relationship!

In [32]:
# Perform crossover
for i in range(0, pop_size, 2):
    if np.random.rand() < crossover_rate:
        parent1 = new_population[i]
        parent2 = new_population[i + 1]

        # Pick a random crossover point
        crosspoint = np.random.randint(1, num_variables)

        # Create two new children
        child1 = np.concatenate((parent1[:crosspoint], parent2[crosspoint:]))
        child2 = np.concatenate((parent2[:crosspoint], parent1[crosspoint:]))

        # Replace the parents with the new children
        new_population[i] = child1
        new_population[i + 1] = child2


Now the new population is like:

In [33]:
new_population

array([[-0.24739641,  1.78155216],
       [-0.24739641,  1.78155216],
       [-0.24739641, -1.2461085 ],
       [ 2.29463618,  1.78155216],
       [-0.24739641,  1.78155216],
       [-0.24739641,  1.78155216],
       [-0.24739641,  1.78155216],
       [-0.24739641,  1.78155216],
       [-0.24739641,  1.78155216],
       [-0.24739641,  1.78155216]])

The survived ones will have the chance to also mutate. Here is the story. Some say that Tyrannosaurus rex especies extincted because they had short hands and couldn't grab the food on the ground! "Some" also say Segnosaurus survived because there was enough food for them but they mutated to develop longer hands and eventually during millions of years (iterations in GA) they turned into monekys. This is not a real story I just made it up!

Anyway, the total number of individuals (or chromosomes) are equal to `pop_size`. The total number of gens in each population holding are chromosomes are `pop_size * num_variables`. And randomly `mutation_rate` percent of them will mutate.

In [34]:
# Calculate the total number of genes
total_gen = pop_size * num_variables

# Calculate the number of genes to mutate
total_gen_mutate = int(total_gen * mutation_rate)

# Generate random indices for genes to mutate
genes_to_mutate_indices = np.random.choice(total_gen, size=total_gen_mutate, replace=False)

print("Total numebr of gens: \n",total_gen);
print("\nTotal number of gens that will randomly mutate: \n",total_gen_mutate);
print("\nThe indexes of gens that will be mutate (numbers between zero to total_gen): \n",genes_to_mutate_indices);


Total numebr of gens: 
 20

Total number of gens that will randomly mutate: 
 6

The indexes of gens that will be mutate (numbers between zero to total_gen): 
 [ 1 17 14 13 15  6]


Here the gens that will mutate are replaced with a random number between lower_bound and upper_bound. These are the bounds for the decision variables (gens).

In [35]:
# Mutate the selected genes
for index in genes_to_mutate_indices:
    row = index // num_variables
    col = index % num_variables
    print(row, col)
    new_population[row, col] = np.random.uniform(low=lower_bound, high=upper_bound)

0 1
8 1
7 0
6 1
7 1
3 0


Now this a new update generation. I recompute the fitness with objective function and the probability of each indiviudal for this new population. I will keep do this until I get enough accuracy.

In [36]:
new_population

array([[-0.24739641, -3.92252144],
       [-0.24739641,  1.78155216],
       [-0.24739641, -1.2461085 ],
       [ 0.90219427,  1.78155216],
       [-0.24739641,  1.78155216],
       [-0.24739641,  1.78155216],
       [-0.24739641,  1.48641848],
       [-0.11100862, -1.89420375],
       [-0.24739641,  1.07954625],
       [-0.24739641,  1.78155216]])

If you wanna see another numerical example you can read:
https://arxiv.org/pdf/1308.4675.pdf